
# Exports

# Experiments:


- Without density

- With density

- With density + gdpcap

- With density + gdpcap + wto + contig

- With density + pci + gdpcap + wto + contig

- With density + pop 

In [1]:
%load_ext autotime
%matplotlib inline

time: 620 ms (started: 2023-07-06 11:31:56 +03:00)


In [2]:
import pandas as pd
import joblib
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import re

time: 1.31 s (started: 2023-07-06 11:31:56 +03:00)


In [3]:
# Reading the data
data =  pd.read_csv('Data/SimpleApproach/Exports/DataExports.csv')
data

,refYear,reporterCode,partnerCode,distw_harmonic,contig,gdp_o,gdp_d,gdpcap_o,gdpcap_d,pop_o,...,wto_o,wto_d,refMonth,partner2Code,cmdCode,primaryValue,2HScmdCode,4HScmdCode,pci,density
0,2016,12.0,4.0,5758.0,0.0,1.560796e+08,1.946902e+07,3.844,0.562,40606.052,...,0.0,1.0,7,0,852352,90.962,85,8523,1.861445,0.010339
1,2016,12.0,36.0,16444.0,0.0,1.560796e+08,1.204616e+09,3.844,49.928,40606.052,...,0.0,1.0,7,0,852352,83.845,85,8523,1.861445,0.010339
2,2016,12.0,40.0,1693.0,0.0,1.560796e+08,3.864278e+08,3.844,44.177,40606.052,...,0.0,1.0,6,0,852352,74.946,85,8523,1.861445,0.010339
3,2016,12.0,51.0,3587.0,0.0,1.560796e+08,1.054733e+07,3.844,3.606,40606.052,...,0.0,1.0,5,0,852352,74.617,85,8523,1.861445,0.010339
4,2016,12.0,56.0,1628.0,0.0,1.560796e+08,4.663657e+08,3.844,41.096,40606.052,...,0.0,1.0,9,0,852352,111.245,85,8523,1.861445,0.010339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102321939,2019,894.0,834.0,1380.0,1.0,2.330867e+07,6.113687e+07,1.305,1.054,17861.034,...,1.0,1.0,8,0,220890,277.000,22,2208,-0.189242,0.054334
102321940,2019,894.0,834.0,1380.0,1.0,2.330867e+07,6.113687e+07,1.305,1.054,17861.034,...,1.0,1.0,8,0,870710,1076.000,87,8707,3.884524,0.027991
102321941,2019,894.0,834.0,1380.0,1.0,2.330867e+07,6.113687e+07,1.305,1.054,17861.034,...,1.0,1.0,8,0,930621,10111.000,93,9306,1.961242,0.031797
102321942,2019,894.0,834.0,1380.0,1.0,2.330867e+07,6.113687e+07,1.305,1.054,17861.034,...,1.0,1.0,9,0,283531,294.000,28,2835,1.207237,0.036452


time: 5min 12s (started: 2023-07-06 11:31:58 +03:00)


In [4]:
def MPE(Y_actual,Y_Predicted):
    mape = np.mean((Y_actual - Y_Predicted)/Y_actual)*100
    return mape

def MAPE(Y_actual,Y_Predicted):
    mpe = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mpe

data['primaryValue'] = data['primaryValue']/1000000
data['gdp_o'] = data['gdp_o']/1000000
data['gdp_d'] = data['gdp_d']/1000000
data['gdpcap_o'] = data['gdpcap_o']/1000000
data['gdpcap_d'] = data['gdpcap_d']/1000000
data['distw_harmonic'] = data['distw_harmonic']/1000

time: 2.83 s (started: 2023-07-06 11:37:10 +03:00)


## RF


In [5]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y


# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 16.0min remaining: 33.9min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 18.3min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 18.8min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    7.5s remaining:  1.4min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    8.5s remaining:    5.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    9.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   31.9s remaining:  6.1min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   35.5s remaining:   23.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   41.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.89
R2 on validation data: 0.86
Mean Squared Error: 8.14
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.86
MAPE:  234652.61982303977
MPE:  -234634.3559379794
time: 20min 35s (started: 2023-07-06 11:37:13 +03:00)


In [5]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y


# DT
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=100, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done  56 out of 100 | elapsed: 60.3min remaining: 47.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 69.8min finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  56 out of 100 | elapsed:   11.4s remaining:    9.0s
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:   16.3s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  56 out of 100 | elapsed:   45.6s remaining:   35.8s
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:  1.0min finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  56 out of 100 | elapsed:   11.6s remaining:    9.1s
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:   16.1s finished


R2 on training data: 0.89
R2 on validation data: 0.87
Mean Squared Error: 7.79
Mean Absolute Error: 0.3
Adjusted R^2 Score: 0.87
MAPE:  263414.36432743387
MPE:  -263396.8421017137
time: 1h 14min 14s (started: 2023-07-03 15:42:45 +03:00)


In [7]:
# +density

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 17.6min remaining: 37.4min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 19.8min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 19.9min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    4.9s remaining:   56.3s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    5.5s remaining:    3.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    6.4s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   18.9s remaining:  3.6min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   21.5s remaining:   14.3s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   24.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.9
R2 on validation data: 0.87
Mean Squared Error: 7.74
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.87
MAPE:  230896.62802950712
MPE:  -230879.00813223686
time: 21min 18s (started: 2023-07-04 12:20:54 +03:00)


In [8]:
# +density

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 24.0min remaining: 51.1min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 24.9min remaining:  4.7min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 25.2min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    4.6s remaining:   52.6s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    5.1s remaining:    3.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   18.5s remaining:  3.5min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   21.2s remaining:   14.1s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   23.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.89
R2 on validation data: 0.86
Mean Squared Error: 8.37
Mean Absolute Error: 0.31
Adjusted R^2 Score: 0.86
MAPE:  274106.83263480366
MPE:  -274089.79541797965
time: 26min 24s (started: 2023-07-04 12:42:13 +03:00)


In [9]:
# +density+gdpcap

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 19.0min remaining: 40.4min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 22.9min remaining:  4.4min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 23.4min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    4.7s remaining:   54.6s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    5.2s remaining:    3.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    6.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   19.5s remaining:  3.7min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   21.5s remaining:   14.3s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   23.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.9
R2 on validation data: 0.87
Mean Squared Error: 7.65
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.87
MAPE:  222723.93105771294
MPE:  -222706.52015010265
time: 25min 1s (started: 2023-07-04 13:08:37 +03:00)


In [10]:
# +density+gdpcap

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 17.0min remaining: 36.2min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 21.1min remaining:  4.0min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 21.2min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.6s remaining:   41.1s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.0s remaining:    2.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   14.8s remaining:  2.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   16.6s remaining:   11.1s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   19.4s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.89
R2 on validation data: 0.86
Mean Squared Error: 8.28
Mean Absolute Error: 0.31
Adjusted R^2 Score: 0.86
MAPE:  268939.8689753926
MPE:  -268922.92018375517
time: 22min 34s (started: 2023-07-04 13:33:38 +03:00)


In [11]:
# +density+gdpcap+pci

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 20.0min remaining: 42.5min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 24.1min remaining:  4.6min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 24.3min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    4.3s remaining:   49.9s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.9s remaining:    3.3s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    5.8s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   17.5s remaining:  3.4min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   19.3s remaining:   12.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   22.9s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.9
R2 on validation data: 0.86
Mean Squared Error: 8.11
Mean Absolute Error: 0.3
Adjusted R^2 Score: 0.86
MAPE:  224965.58391037138
MPE:  -224948.13999064776
time: 25min 51s (started: 2023-07-04 13:56:12 +03:00)


In [6]:
# +density+gdpcap+pci

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 19.0min remaining: 40.5min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 22.6min remaining:  4.3min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 22.8min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.1s remaining:   35.5s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.6s remaining:    2.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   12.7s remaining:  2.4min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   14.7s remaining:    9.8s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   16.0s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.86
R2 on validation data: 0.84
Mean Squared Error: 9.53
Mean Absolute Error: 0.33
Adjusted R^2 Score: 0.84
MAPE:  280746.0859714846
MPE:  -280729.16003705916
time: 24min 5s (started: 2023-07-04 16:14:14 +03:00)


In [5]:
data.columns

Index(['refYear', 'reporterCode', 'partnerCode', 'distw_harmonic', 'contig',
       'gdp_o', 'gdp_d', 'gdpcap_o', 'gdpcap_d', 'pop_o', 'pop_d', 'wto_o',
       'wto_d', 'refMonth', 'partner2Code', 'cmdCode', 'primaryValue',
       '2HScmdCode', '4HScmdCode', 'pci', 'density'],
      dtype='object')

time: 3.36 ms (started: 2023-07-04 12:19:33 +03:00)


In [5]:
# +density+gdpcap+pci + No year

X = data[['distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=15, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 16.8min remaining: 35.6min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 20.0min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 20.1min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.2s remaining:   36.2s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.7s remaining:    2.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   13.7s remaining:  2.6min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   15.5s remaining:   10.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   17.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.85
R2 on validation data: 0.83
Mean Squared Error: 10.38
Mean Absolute Error: 0.34
Adjusted R^2 Score: 0.83
MAPE:  272127.71183696226
MPE:  -272110.65359365626
time: 21min 19s (started: 2023-07-04 15:52:54 +03:00)


In [7]:
# +density+gdpcap+pci + No year

X = data[['distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=15, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 16.1min remaining: 34.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 19.4min remaining:  3.7min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 19.5min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    2.4s remaining:   28.0s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    2.9s remaining:    2.0s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    3.5s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    9.6s remaining:  1.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   11.6s remaining:    7.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   13.3s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.81
R2 on validation data: 0.79
Mean Squared Error: 12.42
Mean Absolute Error: 0.37
Adjusted R^2 Score: 0.79
MAPE:  331849.60827166727
MPE:  -331833.1309516738
time: 20min 40s (started: 2023-07-04 16:53:08 +03:00)


# Log transformations

In [7]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  8.9min remaining: 18.8min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  9.3min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  9.6min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    5.8s remaining:  1.1min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    6.3s remaining:    4.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    7.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   23.4s remaining:  4.5min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   25.5s remaining:   17.0s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   29.2s finished


R2 on training data: 0.37
R2 on validation data: 0.35
MSE:  38.61
MAE:  0.28
R-square:  0.35
MAPE:  11220.78
MPE:  -11146.66
time: 10min 24s (started: 2023-07-05 13:29:48 +03:00)


In [8]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 10.9min remaining: 23.2min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 13.0min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 13.2min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    6.1s remaining:  1.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    6.7s remaining:    4.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    7.8s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   27.0s remaining:  5.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   28.9s remaining:   19.3s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   31.4s finished


R2 on training data: 0.26
R2 on validation data: 0.24
MSE:  45.22
MAE:  0.28
R-square:  0.24
MAPE:  9749.01
MPE:  -9675.23
time: 15min 12s (started: 2023-07-05 13:41:42 +03:00)


In [9]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density', 'gdpcap_o','gdpcap_d']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 12.1min remaining: 25.6min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 15.3min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 15.4min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    6.4s remaining:  1.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    7.3s remaining:    4.8s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    8.2s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   25.8s remaining:  4.9min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   28.8s remaining:   19.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   32.5s finished


R2 on training data: 0.22
R2 on validation data: 0.2
MSE:  47.47
MAE:  0.28
R-square:  0.2
MAPE:  9473.59
MPE:  -9400.11
time: 17min 34s (started: 2023-07-05 13:56:55 +03:00)


In [10]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density', 'gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 16.2min remaining: 34.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 17.0min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 17.4min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    6.6s remaining:  1.3min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    7.1s remaining:    4.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    8.5s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   27.4s remaining:  5.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   29.8s remaining:   19.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   31.5s finished


R2 on training data: 0.21
R2 on validation data: 0.19
MSE:  48.29
MAE:  0.28
R-square:  0.19
MAPE:  9366.37
MPE:  -9292.89
time: 19min 36s (started: 2023-07-05 14:14:29 +03:00)


In [11]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 11.4min remaining: 24.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 14.2min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 14.6min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    6.3s remaining:  1.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    6.9s remaining:    4.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    8.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   25.2s remaining:  4.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   27.6s remaining:   18.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   30.6s finished


R2 on training data: 0.26
R2 on validation data: 0.24
MSE:  45.55
MAE:  0.28
R-square:  0.24
MAPE:  9700.66
MPE:  -9626.85
time: 16min 40s (started: 2023-07-05 14:34:06 +03:00)
